In [1]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

'cuda'

In [2]:
### Word -> Subword
def Word_to_sub(word):
    
    subwords = []
    sublist = ['<'] + [w for w in word] + ['>']
    
    minsub = 2
    maxsub = min(6, len(sublist)-1)
    
    for n_sub in range(minsub, maxsub):
        for n in range(len(sublist)-(n_sub-1)):  #subword의 n-gram에서 n에 따라 subword로 나누기
            subwords.append("".join(sublist[n:n+n_sub]))
    subwords.append("".join(sublist))
    return subwords

In [10]:
import kss
import re
from konlpy.tag import Mecab
import numpy as np

class Prep():
    
    def Clean_text(self, text): # noisy한 문장을 cleaning
        self.sentences = []
        
        for i in range(len(text)):
            if (i+1) % 100 == 0:
                print(str(i+1) + 'th sentence is spliting...')
        
            temp = re.sub("[^가-힣a-z0-9.?]", ' ', text[i]) #한글, 영어, 숫자, 온점, 물음표가 아닌 것을 공백으로 삭제
            temp = re.sub("[.]{2,}", ".", temp) # 반복되는 온점 (...) 삭제
            temp = re.sub("[?]{2,}", "?", temp) # 반복되는 물음표 (?) 삭제
            temp = re.sub("[!]{2,}", "!", temp) # 반복되는 느낌표 (!) 삭제
            temp = re.sub("[' ']{2,}", " ", temp) # 반복되는 공백 삭제 
            temp = kss.split_sentences(temp)  #문장 분리

            for tmp in temp:
                self.sentences.append(tmp)
        
        return self.sentences
        
    def Tokenizer(self, sentences): # cleaned data에 대한 형태소 분석 후 vocabulary 구축
        
        self.corpus = []
        self.words = []
        self.subs = []
        vocab_sub_tmp = []
        
        tokenizer = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")
        
        for sent in sentences:
            temp = tokenizer.morphs(sent) # 각 문장별로 형태소 토큰화
            self.corpus.append(temp)      # corpus에 형태소 추가
            
            for tmp in temp:
                self.words.append(tmp)    # 각 형태소 모으기
                subwords = Word_to_sub(tmp)
                self.subs.append(subwords)   #변환된 Subword들 모으기
                vocab_sub_tmp += subwords   #각 subword리스트 합치기

            self.vocab = set(self.words)    #형태소 vocab
            self.vocab_sub = set(vocab_sub_tmp) #Subword vocab
        
        return self.corpus, self.words, self.subs, self.vocab, self.vocab_sub
    
    def Make_dict(self,vocab, vocab_sub): # 만들어진 Vocabulary를 기준으로 단어-정수 및 정수-단어 인덱싱 dictionary 생성
        self.word_dict = {w: i for i, w in enumerate(vocab)}
        self.index_dict = {i: w for i, w in enumerate(vocab)}
        
        self.subs_dict = {w: i for i, w in enumerate(vocab_sub)}
        self.index_subs_dict = {i: w for i, w in enumerate(vocab_sub)}
        
        return self.word_dict, self.index_dict, self.subs_dict, self.index_subs_dict
    
    def Get_clean(self, text): #위의 세가지 메소드를 한번에 진행                
        sentences =  self.Clean_text(text)
        corpus, words, subs, vocab, vocab_sub = self.Tokenizer(sentences)
        word_dict, index_dict, subs_dict, subs_index_dict = self.Make_dict(vocab, vocab_sub)
        
        return sentences, corpus, words, subs, vocab, vocab_sub, word_dict, index_dict, subs_dict, subs_index_dict

In [4]:
class Make_pairs(): # Input과 target, label을 만들어주는 클래스
    
    def __init__(self):
        
        from collections import defaultdict
        self.inputs = [] 
        self.targets = []
        self.labels = []
        
        # Negative sampling 정의
        self.wordFreq = defaultdict(int)
        for word in words:
            self.wordFreq[word] += 1
        
        self.SumFreq = sum([freq**(3/4) for freq in self.wordFreq.values()]) # 분모
        self.wordProb = {word:((freq**(3/4))/self.SumFreq) for word, freq in self.wordFreq.items()} #샘플링 확률분포
        
    def Draw(self,n_sample): # n_sample : sample을 몇 개를 뽑을건지?
        negsam = []
        for i in range(n_sample):
            negsam.append(np.random.choice(list(self.wordProb.keys()), p=list(self.wordProb.values()))) #확률분포 P에 따른 추출
        
        return negsam
    
    def Get_pairs(self, window_size, n_negsam):
        for c in corpus: # for each sentence          
            for idx in range(len(c)): # for each idx
                start = max(0,idx - window_size)
                tail = min(idx + window_size, len(c)-1)
            
                #add negative samples to context
                context = c[start:idx] + c[idx+1:tail+1]
                needed_negsam = 2*window_size+n_negsam - len(context) # needed number of negative samples
                negsam = self.Draw(needed_negsam) # draw negative samples
            
                #stack pairs
                self.inputs.append(Word_to_sub(c[idx]))
                self.targets.append(context+negsam)
                self.labels.append([1]*len(context) + [0]*needed_negsam)
                  
        return self.inputs, self.targets, self.labels   

In [5]:
from torch.autograd import Variable
def train(inputs, targets, labels, emb_dim, lr, n_epoch):
    
    vocab_size = len(vocab)
    vocab_sub_size = len(vocab_sub)
    target_size = len(targets[0])
    center_mat = Variable(torch.nn.init.xavier_normal_(torch.empty(emb_dim, vocab_sub_size)).cuda(),requires_grad=True).float() # W mat
    context_mat = Variable(torch.nn.init.xavier_normal_(torch.empty(vocab_size, emb_dim)).cuda(),requires_grad=True).float() # W' mat
    
    for epoch in range(n_epoch): 
        loss_value = 0
        for batch in range(len(inputs)):
            
            x = inputs[batch] # word index of input
            x_sub = Variable(torch.FloatTensor([subs_dict[i] for i in x]).cuda())
            h_sub = [center_mat[:,int(i)] for i in x_sub.tolist()] # look-up table
            h = sum(h_sub)
    
            z = torch.matmul(context_mat,h) 
            y = torch.LongTensor(labels[batch]).cuda()

            tar_ind = [word_dict[tar] for tar in targets[batch]]  # make target index                
            context_tmp = Variable(context_mat[tar_ind,:], requires_grad=True).float() # embedded target vectors
            z = torch.matmul(context_tmp,h) 

            y_hat = F.log_softmax(z, dim=0)
            loss = F.nll_loss(y_hat, y)
            loss_value += loss.item()
            loss.backward()
            
            center_mat.data -= lr * center_mat.grad.data # update center_mat
            context_mat.data[tar_ind,:] -= lr * context_tmp.grad.data # update context_mat with context_tmp
            
            # initialize gradient after update
            center_mat.grad.data.zero_() 
            context_tmp.grad.data.zero_()
        
        if epoch % 100 == 0:
            print(f"Loss at this epoch {epoch+100}: {loss_value / vocab_size}")
        
    return center_mat, context_mat         

In [7]:
def TSNEplot(wordvec, vocab, perplexity):
    
    #matplotlib 패키지 한글 깨짐 처리
    import matplotlib.pyplot as plt
    %matplotlib notebook
    plt.rc('font', family='Malgun Gothic') #윈도우, 구글 콜랩
    plt.rcParams['axes.unicode_minus'] = False #한글 폰트 사용시 마이너스 폰트 깨짐 해결

    from sklearn.manifold import TSNE
    
    wordvec_2d = TSNE(n_components=2, perplexity=perplexity, learning_rate='auto', init='random').fit_transform(wordvec)
    plt.scatter(wordvec_2d[:,0], wordvec_2d[:,1], s=0.5)

    test_words=[]
    for i in range(len(vocab)):
        test_words.append(index_dict[i])
    
    for i, word in enumerate(test_words):
        plt.annotate(word, xy=(wordvec_2d[i, 0], wordvec_2d[i, 1]))
    
    return wordvec_2d    

In [ ]:
# Library import
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import re

# 간단한 Corpus
f = open("C:/Users/ImedisynRnD2/Desktop/KTH/Private/NLP_Study/naver_review_train.txt", 'rt', encoding='UTF8')
line = f.readlines()

text = []
for l in line:
    tmp = l.split('\t')[1].split('\t')[0]
    text.append(tmp)
text = text[1:5000]

#parameters
emb_dim = 20
window_size = 2
n_negsam = 10
lr = 0.01
n_epoch = 300

sentences, corpus, words, subs, vocab, vocab_sub, word_dict, index_dict, subs_dict, subs_index_dict = Prep().Get_clean(text) # 전처리
inputs, targets, labels = Make_pairs().Get_pairs(window_size, n_negsam) # Input / target / label 생성
center_mat, context_mat = train(inputs, targets, labels, emb_dim, lr, n_epoch) # Training
# wordvec_2d = TSNEplot(context_mat.data, vocab, perplexity=4) # TNSE plotting

100th sentence is spliting...
200th sentence is spliting...
300th sentence is spliting...
400th sentence is spliting...
500th sentence is spliting...
600th sentence is spliting...
700th sentence is spliting...
800th sentence is spliting...
900th sentence is spliting...
1000th sentence is spliting...
1100th sentence is spliting...
1200th sentence is spliting...
1300th sentence is spliting...
1400th sentence is spliting...
1500th sentence is spliting...
1600th sentence is spliting...
1700th sentence is spliting...
1800th sentence is spliting...
1900th sentence is spliting...
2000th sentence is spliting...
2100th sentence is spliting...
2200th sentence is spliting...
2300th sentence is spliting...
2400th sentence is spliting...
2500th sentence is spliting...
2600th sentence is spliting...
2700th sentence is spliting...
2800th sentence is spliting...
2900th sentence is spliting...
3000th sentence is spliting...
3100th sentence is spliting...
3200th sentence is spliting...
3300th sentence i

In [11]:
## Cos sim v2

import numpy as np
from numpy import dot
from numpy.linalg import norm
 
def cos_sim(word, n=10):
    all_sim = []
    
    subs = Word_to_sub(word) # sub word로 분해
    subs = [i for i in subs if i in list(subs_dict.keys())] # sub_dict에 있는 sub들만 취급
    
    x_sub = [subs_dict[i] for i in subs]
    h_sub = [center_mat[:,i] for i in x_sub] # look-up table
    h = sum(h_sub).cpu().detach().numpy()
    
    # vocab의 단어들과 유사도 비교
    for v in vocab:
        vec = context_mat[word_dict[v],:].cpu().detach().numpy()
        sim = dot(h, vec)/(norm(h)*norm(vec))
        all_sim.append(sim)
    
    top = np.argsort(all_sim)[::-1][:n]
    outword = [index_dict[i] for i in top]
    outsim = [all_sim[t] for t in top]
    
    print('-------------------------------------')
    print('Cosine Similarity of Top %d words' % n)
    print('-------------------------------------')
    for i in range(n):
        print(outword[i] + ': ' + str(outsim[i]))
        
#     return outword, outsim

In [39]:
cos_sim('인공지능채팅')

-------------------------------------
Cosine Similarity of Top 10 words
-------------------------------------
고역: 0.77677375
위원: 0.7241702
유대인: 0.7110779
태어났: 0.654795
눈부신: 0.6399819
과하: 0.62564206
사슴: 0.617252
발연기: 0.614777
빼: 0.6125042
고추: 0.6087969
